In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# experiment 1:
# first, creating a set of N randomly generated (m, 1) vectors:
import numpy as np
m = 5
N = 100
vectors = [np.random.rand(*(m, 1)) for _ in range(N)]

c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# finding nearest neighbor of each vector using O(N^2) brute force approach:
nearest_neighbors = {} # the keys will be the indices of the vectors from 0 to N-1, and the corresponding values will be the indices of its nearest neighbor
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors[i] = None
    for j in range(len(vectors)):
        if j != i:
            dist = np.linalg.norm(vectors[i] - vectors[j])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_neighbors[i] = j
print (nearest_neighbors) 

{0: 44, 1: 67, 2: 60, 3: 20, 4: 12, 5: 43, 6: 84, 7: 24, 8: 21, 9: 82, 10: 77, 11: 87, 12: 60, 13: 80, 14: 42, 15: 93, 16: 65, 17: 36, 18: 59, 19: 90, 20: 90, 21: 98, 22: 88, 23: 91, 24: 42, 25: 0, 26: 43, 27: 67, 28: 92, 29: 93, 30: 78, 31: 44, 32: 59, 33: 43, 34: 0, 35: 83, 36: 17, 37: 97, 38: 40, 39: 35, 40: 38, 41: 73, 42: 46, 43: 33, 44: 0, 45: 33, 46: 42, 47: 85, 48: 63, 49: 54, 50: 54, 51: 1, 52: 69, 53: 26, 54: 49, 55: 42, 56: 35, 57: 19, 58: 78, 59: 32, 60: 12, 61: 84, 62: 8, 63: 48, 64: 97, 65: 16, 66: 70, 67: 1, 68: 88, 69: 52, 70: 52, 71: 12, 72: 25, 73: 41, 74: 89, 75: 54, 76: 23, 77: 10, 78: 69, 79: 67, 80: 13, 81: 94, 82: 9, 83: 35, 84: 6, 85: 98, 86: 39, 87: 56, 88: 68, 89: 13, 90: 19, 91: 23, 92: 28, 93: 15, 94: 81, 95: 4, 96: 55, 97: 37, 98: 85, 99: 97}


In [4]:
# finding nearest neighbor of each vector using NaiveLSH:
from memristor.engine.model import NaiveLSH
from memristor.crossbar.model import LineResistanceCrossbar
from memristor.devices import StaticMemristor
naive_lsh = NaiveLSH(
    hash_size=3, # adjustable hyperparameter
    crossbar_class=LineResistanceCrossbar,
    crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
    memristor_model_class=StaticMemristor,
    memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
    m=m,
    r=1, # adjustable hyperparameter
)
reps = 5 # adjustable hyperparameter (repetitions of the hashing)
all_bins = []
for _ in range(reps):
    bins = {}
    for i in range(len(vectors)):
        hash = naive_lsh.inference(vectors[i])
        if hash not in bins.keys():
            bins[hash] = [i]
        else:
            bins[hash].append(i)
    for bin in list(bins.values()):
        all_bins.append(bin)
#      {010:[1,5,7], 111:[5,6,7]}

# now at this point all_bins is a list like [[1,2], [1,3,5], ... ] where each element of all_bins is a bin containing indices of vectors that are likely
# to be close to each other. so now to find the nearest neighbor for each vector, we simply iterate through and check only those vectors that share a bin
# with it, so in this case for 1 we would check 2, 3, and 5, to find the nearest neighbor
nearest_neighbors_approx = {}
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors_approx[i] = None
    for bin in all_bins:
        if i in bin:
            for j in bin:
                if j != i:
                    dist = np.linalg.norm(vectors[i] - vectors[j])
                    if dist < nearest_dist:
                        nearest_dist = dist 
                        nearest_neighbors_approx[i] = j
print (nearest_neighbors_approx)

c:\Users\Ahsan\OneDrive - University of Toronto\Desktop\onedrive_ruined_everything\pytorch-metal-oxide-memristor-crossbar\memristor\crossbar\model.py:32: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  for i in range(ideal_w.shape[0])]).squeeze()


{0: 44, 1: 67, 2: 60, 3: 20, 4: 12, 5: 43, 6: 84, 7: 24, 8: 21, 9: 82, 10: 77, 11: 87, 12: 60, 13: 80, 14: 42, 15: 93, 16: 65, 17: 36, 18: 59, 19: 90, 20: 90, 21: 98, 22: 88, 23: 91, 24: 42, 25: 0, 26: 43, 27: 67, 28: 92, 29: 93, 30: 78, 31: 44, 32: 59, 33: 43, 34: 0, 35: 83, 36: 17, 37: 97, 38: 40, 39: 35, 40: 38, 41: 73, 42: 46, 43: 33, 44: 0, 45: 33, 46: 42, 47: 85, 48: 63, 49: 54, 50: 54, 51: 1, 52: 69, 53: 26, 54: 49, 55: 42, 56: 35, 57: 19, 58: 78, 59: 32, 60: 12, 61: 84, 62: 8, 63: 48, 64: 97, 65: 16, 66: 70, 67: 1, 68: 88, 69: 52, 70: 52, 71: 12, 72: 25, 73: 41, 74: 89, 75: 54, 76: 23, 77: 10, 78: 69, 79: 67, 80: 13, 81: 94, 82: 9, 83: 35, 84: 6, 85: 98, 86: 39, 87: 56, 88: 68, 89: 13, 90: 19, 91: 23, 92: 28, 93: 15, 94: 81, 95: 4, 96: 55, 97: 37, 98: 85, 99: 97}


In [5]:
print (nearest_neighbors)
print (nearest_neighbors_approx)

{0: 44, 1: 67, 2: 60, 3: 20, 4: 12, 5: 43, 6: 84, 7: 24, 8: 21, 9: 82, 10: 77, 11: 87, 12: 60, 13: 80, 14: 42, 15: 93, 16: 65, 17: 36, 18: 59, 19: 90, 20: 90, 21: 98, 22: 88, 23: 91, 24: 42, 25: 0, 26: 43, 27: 67, 28: 92, 29: 93, 30: 78, 31: 44, 32: 59, 33: 43, 34: 0, 35: 83, 36: 17, 37: 97, 38: 40, 39: 35, 40: 38, 41: 73, 42: 46, 43: 33, 44: 0, 45: 33, 46: 42, 47: 85, 48: 63, 49: 54, 50: 54, 51: 1, 52: 69, 53: 26, 54: 49, 55: 42, 56: 35, 57: 19, 58: 78, 59: 32, 60: 12, 61: 84, 62: 8, 63: 48, 64: 97, 65: 16, 66: 70, 67: 1, 68: 88, 69: 52, 70: 52, 71: 12, 72: 25, 73: 41, 74: 89, 75: 54, 76: 23, 77: 10, 78: 69, 79: 67, 80: 13, 81: 94, 82: 9, 83: 35, 84: 6, 85: 98, 86: 39, 87: 56, 88: 68, 89: 13, 90: 19, 91: 23, 92: 28, 93: 15, 94: 81, 95: 4, 96: 55, 97: 37, 98: 85, 99: 97}
{0: 44, 1: 67, 2: 60, 3: 20, 4: 12, 5: 43, 6: 84, 7: 24, 8: 21, 9: 82, 10: 77, 11: 87, 12: 60, 13: 80, 14: 42, 15: 93, 16: 65, 17: 36, 18: 59, 19: 90, 20: 90, 21: 98, 22: 88, 23: 91, 24: 42, 25: 0, 26: 43, 27: 67, 28: 